# PCA

[PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) un modelo de Aprendizaje No-Supervisado que se utiliza para la reducción de la dimensionalidad. A diferencia de los modelos No Supervisados vistos anteriormente, no buscamos agrupar los datos, sino transformarlos. Reorganizaremos la información de manera tal que no tendremos más las variables que tenemos actualmente, sino que las desarmamos y generamos componentes.

Al igual que los modelos de aprendizaje no-supervisado vistos anteriormente, tiene los mismos pasos que el aprendizaje supervisado salvo por dos cuestiones: no se realiza la división de datos ni tampoco la evaluación.

**No se realiza la división de los datos**, ya que no tenemos una variable a predecir **y** por lo cual no es necesario realizar esa división, tampoco se realiza la división en entrenamiento y testo porque no hay una predicción "correcta", sino solamente la forma de organizar los datos

En este caso si es necesario utilizar el método *.transform* para transformar los datos luego de entrenar el modelo

Por lo tanto los pasos que realizaremos son:
1. Definición del Problema
2. Búsqueda de datos 
3. Exploración y Limpieza de Datos
4. Entrenamiento del modelo
5. Transformar los datos

##### Problema y Búsqueda de datos

En primer lugar utilizaremos el Dataset de las Flores *Iris* visto en una clase anterior con el objetivo de poder graficarlo en 2 dimensiones. El Dataset está extraído de [Kaggle](https://www.kaggle.com/uciml/iris)

Luego utilizaremos otro ejemplo para visualizar los que sucede cuando quitamos componentes.


In [ ]:
#importamos las librerías que utilizaremos

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("Iris.csv")

In [ ]:
# Vemos los primeros 3 registros

data.head(3)

#### Datos que utilizaremos para entrenar

Quitamos en este ejemplo las columnas *Id* que no tiene utilizada y la columna *Species* que es la variable a predecir que no será parte de la reducción de la dimensionalidad, solo lo realizaremos con las variables predictoras que son numéricas.

In [ ]:
# Creamos X que utilizaremso para realizar la reducción de dimensaionalidad

X = data.drop(columns = ["Id", "Species"])

In [ ]:
X.head(3)

In [ ]:
# Importamos el modelo PCA

from sklearn.decomposition import PCA


In [ ]:
# Instanciamos el modelo PCA - Hiperparametro por default: todos los componentes principales

pca = PCA()

In [ ]:
# Rntrenamos el modelo

pca.fit(X)

In [ ]:
# Vemos los componentes

pca.components_

In [ ]:
# T es para cambiar el sentido de los datos
componentes = pca.components_.T
componentes.shape

In [ ]:
#Creamos un DataFrame con los datos de los componentes para ver la relación con las variables originales

pca_componentes = pd.DataFrame(componentes.T, index=X.columns, columns=['PC1', 'PC2', 'PC3', 'PC4'])
pca_componentes

#### Transformación de los datos

Ya hemos entrenado el modelo, pero si queremos transformar los datos y crear un nuevo dataset debemos utilizar el método de la librería Scikit-Learn: fit_transform (que entrena y transforma los datos en la misma linea de código)

In [ ]:
# Transformamos los datos originales con el entrenamiento del modelo

data_pca = pca.fit_transform(X)

In [ ]:
# Creamos un DataFrame con los datos transformados poniendo como indice la especie

data_pca = pd.DataFrame(data_pca, columns=['PC1', 'PC2', 'PC3', 'PC4'],index=data["Species"])
data_pca.head(5)

In [ ]:
features = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
#data.columns

#### Visualización de los datos

Realizaremos una visualización en 2 dimensiones con los componentes generados como resultado de nuestro entrenamiento, es decir en vez de usar las 4 variables que tenemos en el dataset original, utilizaremos los dos componenetes genereados como *ejes* *x* e *y* para poder realizar la visualización. Las *Species* se mantienen.

En segundo lugar incluiremos en los gráficos como están formados los componenetes en base a las variables originales.

In [ ]:
#Graficaremos los datos del dataset Iris en 2 dimensiones

plt.subplots(figsize = (10, 10))
sns.scatterplot(data = data_pca, x = "PC1", y = "PC2", s=100, hue=data_pca.index.tolist())


In [ ]:
plt.subplots(figsize = (7, 7))

# Hacemos un scatter de los datos en las dos primeras componentes
sns.scatterplot(data = data_pca, x = "PC1", y = "PC2", s=100, hue=data_pca.index.tolist())

# Ploteamos las líneas de referencia
#plt.hlines(0,-3.5,3.5, linestyles='dotted', colors='grey')
#plt.vlines(0,-3.5,3.5, linestyles='dotted', colors='grey')

# Hacemos el grafico de las flechas indicando las direcciones de los features originales

# Recorremos cada feature
for i in range(len(features)):

  # Creamos una flecha que vaya del origen y apunte en la dirección de los features
  plt.arrow(0, 0, componentes.T[i][0], componentes.T[i][1], width = 0.05, alpha = 0.5)

  # Indicamos con texto a qué feature corresponde cada flecha
  plt.text(componentes.T[i][0], componentes.T[i][1], s = features[i], fontdict= {'color': 'k', 'size': 10})

plt.xlabel('Primer componente principal')
plt.ylabel('Segunda componente principal')


## PCA Caras

Para poder tener otra visualización de lo que sucede al reducir la dimensionalidad y la pérdida o no de información que esto implica, utilizaremos un dataset de Caras importado directamente de la librearía Scikit-Learn. 

Cada una de las caras está descrito por un vector de 4096 píxeles. El Dataset está compuesto por 400 caras de 4096 features.

El código es un poco más complejo pero vamos a centrarnos en las imágenes y las diferencias al modificar la cantidad de componentes que tenemos.

Esta clase está basada en una Clase del Laboratorio de Datos - Facultad de Exactas - UBA http://materias.df.uba.ar/lda2021c1/171-2/


In [ ]:
# Importamos el dataset de Scikit-Learn

from sklearn.datasets import fetch_olivetti_faces # para cargar el dataset de caras
data, targets = fetch_olivetti_faces(return_X_y = True) # cargamos las caras

data.shape

##### Visualización de Caras

Vamos a visualizar algunas caras al azar para ver como esta compuesto el dataset.

In [ ]:
import numpy as np

# Ploteo 25 imagenes al azar
fig = plt.figure(figsize = (8,8)) # seteo el tamano de la figura
for i in range(25):
    j = np.random.randint(0, data.shape[0]) # en cada iteracion elijo un numero random entre 0 y la longitud de train_raw 
    plt.subplot(5,5,i+1) # Voy a tener una matriz de 5x5 subplots y voy llenando en la iteracion i-esima el subplot i+1
    plt.imshow(data[j,:].reshape(64,64), interpolation='none', cmap="gray") # plotea una imagen random, pues es la imagen j-esima del set de entrenamiento, en formato (28,28) para imagenes en escala de grises (tengo que reshapear)
    plt.title("Persona: {}".format(targets[j]), fontsize = 10) # pongo el titulo a los plots con el identificador unico de la persona 
    plt.xticks([]) # le saco los ticks en el eje X
    plt.yticks([]) # le saco los ticks en el eje Y
plt.show()


#### Entrenamiento del modelo de PCA

Vamos a realizar la reducción de dimensionalidad con 2 componentes para poder visualizar el dataset completo. La cantidad de componenentes es un hiperparámetro a definir al momento de instanciar el modelo

In [ ]:
# Intanciamos el modelo de PCA con 100 componentes
pca = PCA(n_components =2)

In [ ]:
# Entrenamos los datos
pca.fit(data)

In [ ]:
# Transformamos los datos 

data_pca = pca.transform(data)

# (esto puede ser realizado en una sola linea con el método .fit_transform)

In [ ]:
# Creamos un DataFrame con los dos componentes principales solo para poder comprenderlo

df_pca = pd.DataFrame(data_pca, columns=['PC1', 'PC2'])
df_pca.head(5)

##### Visualización del dataset en 2 dimensiones

Ahora que hemos reducido la dimensionalidad podemos graficar el dataset en 2 dimensiones, cada punto será una cara y el lugar que ocupa en el gráfico dependerá de las características que tiene. Luego observaremos algunas caras cercanas y alejadas para ver si encontramos un criterio.

In [ ]:
# Visualizamos nuestro dataset en 2 dimensiones

fig, ax = plt.subplots(figsize = (10, 10))
ax.scatter(data = df_pca, x = "PC1", y = "PC2")

# Por cada dato escribimos a qué instancia corresponde
for i in range(data.shape[0]):
   ax.text(data_pca[i, 0], data_pca[i, 1], s = i)

In [ ]:
# Podemos visualizar alguna cara en particular

fig = plt.figure(figsize = (4,4)) # seteo el tamano de la figura
plt.imshow(data[79,:].reshape(64,64), interpolation='none', cmap="gray") # plotea una imagen random, pues es la imagen j-esima del set de entrenamiento, en formato (28,28) para imagenes en escala de grises (tengo que reshapear)
plt.xticks([]) # le saco los ticks en el eje X
plt.yticks([]) # le saco los ticks en el eje Y
plt.show()

#Ver distintos números y como son más diferentes o más parecidos de acuerdo al lugar en el espacio!!

### PCA con 100 componentes

Realizaremos la reducción de dimensionalidad con 100 componentes para observar cuánto se pierde en variabilidad y a su vez cómo cambia la resolución a medida que quitamos componentes

In [ ]:
# Intanciamos el modelo de PCA con hiperparametro 100 componentes
pca_100 = PCA(n_components=100)

In [ ]:
# Entrenamos el modelo PCA con 100 componentes

pca_100.fit(data)

In [ ]:
# Transformamos los datos

data_pca = pca_100.transform(data)
data_pca

#### Varianza

Es posible observar la variabilidad que aporta cada componente a los datos utilizando el atributo *explained_variance_ratio_* de Scikit Learn que indica el porcentaje de varianza explicada por cada uno de los componentes seleccionados.

* la primera componente principal explica el 24% de la varianza de los datos

* la segunda componente principal explica el 14% de la varianza de los datos

* la tercera componente principal explica el 0.08% de la varianza de los datos

* la cuarta componente principal explica el 0.04% de la varianza de los datos
...

In [ ]:
pca_100.explained_variance_ratio_

In [ ]:
# Varianza 

sns.lineplot(x = range(len(pca_100.explained_variance_ratio_)), y = pca_100.explained_variance_ratio_)
plt.ylabel('Fracción de varianza explicada')
plt.xlabel('Número de componentes principal')

In [ ]:
# Calculamos el acumulado con la función cumsum de numpy para poder graficar la acumulación 
 
varianza_acumulada = np.cumsum(pca_100.explained_variance_ratio_)

varianza_acumulada

In [ ]:
# Graficamos la varianza acumulada
sns.lineplot(x = range(len(pca_100.explained_variance_ratio_)), y = np.cumsum(pca_100.explained_variance_ratio_))
plt.ylabel('Fracción de varianza explicada')
plt.xlabel('Número de componentes principal')

In [ ]:
# Visualizamos en un mismo gráfico

plt.figure(figsize=(7,7))
plt.plot(pca_100.explained_variance_ratio_,  '-o', label='Componente individual')
plt.plot(np.cumsum(pca_100.explained_variance_ratio_), '-s',label='Acumulado')
plt.ylabel('Porcentaje de Varianza Explicada')
plt.xlabel('Componentes Principales')


Si bien los datos tienen 4096 variables (pixeles), se alcanza el 90% de la información con 60 componentes principales, es decir que se reduce la dimensionalidad sin una pérdida significativa de información.

#### Visualización de caras cambiando componentes principales

Vamos a comparar una misma cara como se ve a medida que quitamos componentes

In [ ]:
# Elegimos una cara de ejemplo para ver como se ve original
figura = 10

plt.imshow(data[figura, :].reshape(64, 64), interpolation='none', cmap="gray") # plotea la imagen de índice faceid en formato (28,28) para imagenes en escala de grises (tengo que reshapear)
#plt.xticks([]) # le saco el eje X
#plt.yticks([]) # le saco el eje Y

Visualizamos la cara luego de realizar la reducción de dimensionalidad a 100 componentes utilizando el atributo *inverse_transform* de Sckit-Learn que transforma la data de nuevo a su espacio original. 

In [ ]:
# Repetimos la cara seleccionada
figura = 10

# Reconstrucción de la cara desde el espacio reducido!!!
X_r = pca_100.inverse_transform(data_pca)


plt.imshow(X_r[figura, :].reshape(64, 64), interpolation='none', cmap="gray") # plotea la imagen de índice faceid en formato (28,28) para imagenes en escala de grises (tengo que reshapear)
#plt.xticks([]) # le saco el eje X
#plt.yticks([]) # le saco el eje Y

In [ ]:
# Vamos a realizar la misma acción pero cambiando la cantidad de componentes. Ejemplo: 200, 50, 1, 400!!

#instanciamos el modelo cambiando la cantidad de componenetes
pca = PCA (n_components=200)

#entrenamos el modelo y transformamos los datos
data_pca = pca.fit_transform(data)

# Reconstruimos la cara con el espacio reducido
pca_inverse = pca.inverse_transform(data_pca)

#Visualizamos
plt.imshow(pca_inverse[figura, :].reshape(64, 64), interpolation='none', cmap="gray") # plotea la imagen de índice faceid en formato (28,28) para imagenes en escala de grises (tengo que reshapear)
#plt.xticks([]) # le saco el eje X
#plt.yticks([]) # le saco el eje Y

Cuanto más componentes principales, mejor será la reproducción de la cara original. Igualmente con pocas componentes (con muchas menos que la cantidad de features del espacio original) ya se pueden ver imagenes parecidas a la original.